In [36]:
#This cell creates the csv file to be used in the entire project
import pandas as pd
import csv

data=[]

#Read text file to extract data
with open("./PathLinker_2018_human-ppi-weighted-cap0_75.txt",'r') as data_file:
    for line in data_file:
        #Divide the data by whitespace into 4 columns
        appended = line.split(maxsplit=3)

        #Data is a list of lists of strings, where each string represents a cell in the text file
        data.append(appended)

tail = []
head = []
edge_weight = []
edge_type=[]

#Append each column in a list
for i in data:
   #Clean column names for convenience
   tail.append(i[0].strip("#"))
   head.append(i[1])
   edge_weight.append(i[2])
   edge_type.append(i[3].strip())

#Zip the 4 columns into rows
rows = zip(tail, head, edge_weight, edge_type)

#Write the rows in a csv file
with open("./PathLinker_2018_human-ppi-weighted-cap0_75.csv", "w", newline="") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


In [37]:
#This cell lists all directly connected proteins to a picked protein

import numpy as np
df = pd.read_csv("./PathLinker_2018_human-ppi-weighted-cap0_75.csv")

#Define dataframe columns
tail = df['tail']
head = df['head']
edge_weight = df['edge_weight']
edge_type = df['edge_type']

#Find the most common protein in all PPIs 
most_common_protein=df.mode()
if(most_common_protein['tail'][0]==most_common_protein['head'][0]):
    protein = most_common_protein['tail'][0]
else:
    protein = "P05067"

#Extract the rows where said protein is a tail or head into a new dataframe
protein_df = df[df.eq(protein).any(axis=1)]
protein_df=protein_df.iloc[:,:3]

#Remove the protein name from the dataframe for better readability
protein_list=protein_df.values.tolist()
for row in protein_list:
    if(row[0]==protein):
        row.append("Out")
    if(row[1]==protein):
        row.append("In")
    row.remove(protein)
    
#Save the list of rows as a dataframe
modified_protein_df = pd.DataFrame(protein_list)

#Save the first resulting dataframe into a text file
np.savetxt(r'./protein_connections.txt', modified_protein_df, fmt='%s')

#Calculate in and out degrees of the picked protein
in_degree = len(protein_df.loc[protein_df['head'] == protein])
out_degree = len(protein_df.loc[protein_df['tail'] == protein])

#Append the in and out degrees of the protein into the protein_connections.txt file
with open("./protein_connections.txt", "a") as f:
    f.write("In degree: ")
    f.write(str(in_degree) + "\n")
    f.write("Out degree: ")
    f.write(str(out_degree) + "\n")